## Predicting S1 & T1 Energies

In this notebook, the model run on the supercomputer is loaded and used to predict S1 and T1 excited state energies in TADF molecules


In [2]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from torcheval.metrics import R2Score

from rdkit import Chem
from rdkit.Chem import AllChem

import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm

import pandas as pd
import numpy as np

import random

from sklearn.metrics import mean_absolute_error, mean_squared_error

In [3]:
SEED = 23

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

In [4]:
class MFDataset(Dataset):

    def __init__(self, fn, length = None):
        # Data Loading
        loaded_data = pd.read_csv(fn, sep ="\t", header=None)
        all_data = loaded_data[:length]
        self.data = all_data #Return all data as dataframe

        #Manipulate data using Pandas & RDkit
        all_data.columns = ["ID", "SMILES","LUMO", "HOMO", "E(S1)", "E(T1)"]
        filt_data = all_data.drop(columns = ["ID", "LUMO", "HOMO"])
        filt_data["MOL"] = filt_data["SMILES"].apply(lambda x: Chem.MolFromSmiles(x)) #Add column of Molecular objects

        def calculate_MFP(molecule):
            fp = AllChem.GetMorganFingerprintAsBitVect(molecule, 3, nBits=1024)
            nf = fp.ToList()
            return nf
        
        filt_data["MFP"] = filt_data["MOL"].apply(calculate_MFP)

        mfps = np.array(filt_data["MFP"].tolist())
        #e_s1 = np.array(filt_data["E(S1)"])
        #e_t1 = np.array(filt_data["E(T1)"])

        energies = np.column_stack((filt_data["E(S1)"], filt_data["E(T1)"]))
        
        self.mfps = mfps #Vector of Morgan fingerprints (X by 1024)
        self.energies = energies # Matrix of S1 & T1 energies(X by 2)
        self.n_samples = filt_data.shape[0] #number of data_points

    def __getitem__(self, index):
        # dataset[0]

        return self.mfps[index], self.energies[index]

    def __len__(self):
        # Length of Dataset
        return self.n_samples

In [5]:
path = "./data/TADF_data_DL.txt" #location of data

full_dataset = MFDataset(path)

#Splitting dataset 8:1:1
total_size = len(full_dataset)
train_size = int(0.8*total_size)
validation_size = int(0.1*total_size)
test_size = total_size - (train_size + validation_size)

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(full_dataset, [train_size, validation_size, test_size])

#Create DataLoaders for training, validation, & testing
bs = 32

train_dataloader = DataLoader(train_dataset, batch_size = bs, shuffle = True)
val_dataloader = DataLoader(val_dataset, batch_size = bs, shuffle = False)
test_dataloader = DataLoader(test_dataset, batch_size = bs, shuffle = False)

In [6]:
#Layers building block
class MLP(nn.Module):
    def __init__(self, in_feature, out_feature, dropout):
        super().__init__()
        self.in_feature = in_feature
        self.dropout = dropout
        self.Linear=nn.Linear(in_feature,out_feature)
        self.dropout = nn.Dropout(p=self.dropout)
        self.activation = nn.ReLU()

    def forward(self, x):
        skip_x = x
        x = self.Linear(x)
        x = self.dropout(x)
        x = x+skip_x
        x = self.activation(x)

        return x

#Fingerprint MLP model
class FpMLP(nn.Module):
    def __init__(self, args):
        super(FpMLP, self).__init__()

        # Argument Define
        self.dim_of_fp = args["fp_dim"]
        self.dim_of_Linear = args["hidden_dim"]

        self.N_predict_layer = args["N_MLP_layer"]
        self.N_predict_FC = args["N_predictor_layer"]

        self.N_properties = args["N_properties"]

        self.dropout = args["dropout"]

        self.embedding=nn.Linear(self.dim_of_fp,self.dim_of_Linear)

        self.MLPs= nn.ModuleList([
            MLP(self.dim_of_Linear,self.dim_of_Linear,self.dropout) for _ in range(self.N_predict_layer)])

        self.predict = \
            nn.ModuleList([
                nn.Sequential(nn.Linear(self.dim_of_Linear,self.dim_of_Linear),
                              nn.Dropout(p=self.dropout),
                              nn.ReLU())
                for _ in range(self.N_predict_FC-1)] +
                [nn.Linear(self.dim_of_Linear,self.N_properties)
            ])

    def forward(self, x):
        x = self.embedding(x)

        for layer in self.MLPs:
            x = layer(x)

        for layer in self.predict:
            x = layer(x)
        return x

In [10]:
#Evaluation Loop
#Similar to training loop but gradients are not calculated (with torch.no_grad() uses less memory)
def evaluate(model, iterator, loss_funct):

    epoch_loss = 0
    r2_score = 0
    model.eval()

    maes = 0
    mses = 0

    with torch.no_grad():
        for (x, y) in tqdm(iterator, desc="Evaluating", leave=True):
            x = x.to(torch.float32) #MFP
            y = y.to(torch.float32) #S1 & T1 energies

            y_pred = model(x)
            loss = loss_funct(y_pred, y)

            mae = mean_absolute_error(y.numpy(), y_pred.numpy())
            maes += mae

            mse = mean_squared_error(y.numpy(), y_pred.numpy())
            mses += mse

            r2 = R2Score()
            r2.update(y_pred,y)
            r2_score += float(r2.compute())

            epoch_loss += loss.item()

    return epoch_loss / len(iterator), r2_score / len(iterator), maes / len(iterator), mses / len(iterator)

The model and its configurations (hyperparemeters, loss function, & learning rate) are loaded using the function below. The configurations used in training the model are also displayed.

In [8]:
def load_model(filename):
    checkpoint = torch.load(filename)
    config = checkpoint["config"]
    
    # Create an instance of your model using the configuration
    model = FpMLP(config)
    
    model.load_state_dict(checkpoint["model_state_dict"])
    model.eval()
    
    return model, config

loaded_model, config = load_model("./models/model_4.pt")

config

{'fp_dim': 1024,
 'hidden_dim': 1869,
 'N_MLP_layer': 7,
 'N_predictor_layer': 1,
 'N_properties': 2,
 'dropout': 0,
 'learning_rate': 0.0003108135902739477,
 'criterion': SmoothL1Loss()}

In [12]:
test_loss, test_r2, test_mae, test_mse = evaluate(loaded_model, test_dataloader, config["criterion"])

print("")
print(f"Loss on test set:  {test_loss:.4f}")
print(f"MSE on test set:  {test_mse:.4f}")
print(f"MAE on test set:  {test_mae:.4f}")
print(f"R2 score on test set:  {test_r2:.4f}")

Evaluating: 100%|██████████| 132/132 [00:05<00:00, 23.71it/s]


Loss on test set:  0.0685
MSE on test set:  0.1415
MAE on test set:  0.2706
R2 score on test set:  0.6208


The model, trained using the same configurations as in the literature, achieves an MAE score of 0.2706 eV on the test set. To showcase model performance of a specific molecule, the following code can be employed:

In [14]:
data_1_x = test_dataset[0][0] #First test MFP
data_1_x = torch.tensor(data_1_x, dtype = torch.float32) #Convert to tensor of appropriate data type

# Pass the input feature through the model to get the predicted output
with torch.no_grad():
    predicted_output = loaded_model(data_1_x.unsqueeze(0))  # Unsqueeze to add batch dimension

# You can now use the predicted output for further analysis or comparison
print("Predicted Energies:", predicted_output.numpy().flatten())
print("Actual Energies: ", test_dataset[0][1])

Predicted Energies: [3.4666114 2.7837038]
Actual Energies:  [3.416  2.8756]


In [ ]:
2aa9cdcc63ea380134092a280d164913100487c8